In [ ]:
# import argparse
import os, sys

import awkward as ak

import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use(hep.style.CMS)

from matplotlib.colors import LogNorm

# import uproot

import numpy as np
import pandas as pd

from glob import glob

In [ ]:
plt.rcParams["figure.facecolor"] = "white"

# compare masks

In [ ]:
! ls /eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano_DT12x/rate_tables/*.parquet

In [ ]:
# fname = "~/cernbox/SWAN_projects/L1T-Ph2-Menu/rates/menu/outputs/new_fwk_rates/V29_fromCache_Full_wBtag/rates_full_wBtag_V29_masks.parquet"
# fname = "/eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano/rate_tables/rates_full_Final_V38nano_masks.parquet"
# fname = "/eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano_DT12x/rate_tables/menu_Step1_tkMuVLoose_V38nano_DT12x_masks.parquet"
# fname = "/eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano_DT12x/rate_tables/menu_Step1_V38nano_DT12x_masks.parquet"
fname = "/eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano_DT12x/rate_tables/rates_Step1and2_V38nano_DT12x_masks.parquet"

In [ ]:
new_masks = ak.from_parquet(fname)

In [ ]:
new_masks

In [ ]:
%%time
# new fwk
df_masks = ak.to_dataframe(new_masks)

### Pure

In [ ]:
%%time
test_seed = "L1_DoublePFJet_dEtaMax"

or_all_excl_seed = False

for seed in new_masks.fields:
    if seed == test_seed:
        continue

    or_all_excl_seed = or_all_excl_seed | new_masks[seed]

In [ ]:
np.sum((or_all_excl_seed == False) & new_masks[seed])

In [ ]:
np.sum(new_masks[seed])

In [ ]:
np.sum(or_all_excl_seed)

In [ ]:
df_masks[seed].sum()

In [ ]:
%%time

counts = {}

for seed in df_masks.columns:
    counts[seed] = {
        "total": df_masks[seed].sum(),
        "pure": (
            (df_masks[seed] == True) & ~(df_masks.drop(seed, axis=1).any(axis=1))
        ).sum(),
    }

df_counts = pd.DataFrame(counts).T

In [ ]:
df_counts.sort_values("total", ascending=False)

In [ ]:
df_counts.sort_values("total", ascending=False).plot.barh(figsize=(10, 20))

In [ ]:
ax = df_counts.sort_values("total", ascending=False).plot.barh(figsize=(10, 20))
ax.set_xscale("log")

In [ ]:
sel = df_counts.index.str.contains("Mu")
df_counts[sel].sort_values("total", ascending=False).plot.barh(figsize=(10, 10))

In [ ]:
# df_counts

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(20, 16), sharey=True)
# hep.cms.label(ax=axs[0], llabel="Phase-2 Simulation", rlabel = "14 TeV")
# hep.cms.label(ax=axs[0], llabel="Phase-2 Simulation")
# hep.cms.label(ax=axs[2], label = "14 TeV")

ax1, ax2, ax3 = axs
rate_fact = 11.2 * 2700 / len(df_masks)
df = df_counts.sort_values("pure") * rate_fact
df.plot(kind="barh", ax=ax1)
df.plot(kind="barh", ax=ax2)

ax1.set_xlabel("L1 Rate [kHz]")
ax1.grid()

ax2.set_xscale("log")
ax2.set_xlabel("L1 Rate [kHz]")
ax2.grid()

# pure/total
(df.pure / df.total).plot(kind="barh", ax=ax3, legend=False, color="C2")

# ax3.set_xscale("log")
ax3.set_xlabel("Pure/Total")
ax3.grid()


plt.subplots_adjust(wspace=0, hspace=0)

plt.tight_layout()

for ext in [".png", ".pdf"]:
    outfname = fname.replace(".parquet", f"_pureRates{ext}")
    plt.savefig(outfname)

In [ ]:
outfname

In [ ]:
! ls /eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano_DT12x/rate_tables/menu_Step1_V38nano_DT12x_masks_pureRates.png

In [ ]:
! readlink -f /eos/home-a/alobanov/www/L1T/Phase2/menu/Validation/NewMenuTools/V38nano_DT12x/rate_tables